In [1]:
import os
os.getpid()

13223

In [1]:
import random
list2 = []
#file_data = open('amazon_review_less_than_300_chars_balanced.csv')
with open('balanced_data.csv') as file_data:
    for i in range(5000):
        list2.append(file_data.readline())
    '''for row in file_data:
        list2.append(row)## Data pre-processing module'''
    
list3 = []
'''for i in range(len(list2)):
    list3.append((list2[i][-2],list2[i][:-3]))'''
for line in list2:
    list3.append((line[-2], line[:-3]))
    
random.seed(10)
random.shuffle(list3)

lenth = len(list3)
train_list = list3[0:int(lenth*0.8)]
test_list = list3[int(lenth*0.8):]
print(len(test_list), len(train_list))
#print(train_list[3000])

l = []
for i,j in list3:
    if i not in l:
        l.append(i)
print(l)
di = {}
test_list[0][0]
for i in test_list:
    if i[0] not in di.keys():
        di.update({i[0]:0})
    else:
        di[i[0]] += 1
print(di)

1000 4000
['3', '4', '5', '2', '1']
{'5': 202, '3': 196, '4': 203, '1': 188, '2': 206}


In [2]:

import numpy as np

import torch
from torch.utils.data.sampler import BatchSampler, Sampler
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import random
from tqdm import tqdm

SEED=4321
random.seed(SEED)
np.random.seed(SEED)


class Corpora():
    """
    The class holds training and test corpora.
    """

    def __init__(self):
        """
        Constructor
        """
        # word to index (1-based integers) mapping
        self.word_index = {"N0O0N":0}
        # list of reviews tuples, each of which is (sentence_list, rate),
        self.training_reviews = []
        # (sentence_list, rate) Same format as training_sentences
        self.test_reviews = []

        self.max_len = 0


    # input: a tuple (reviewList, rate)
    # todo: insert values into fields
    # Return the list representing all index of words in a review.
    def insert_fields(self, input):   
        # Sentence list
        word_indexes = []
        for word in input:
            if word not in self.word_index:
                self.word_index[word] = len(self.word_index)
                #self.word_index.update({word:len(self.word_index.keys())}) #No add 1 because 0 is already in
            # find the index of this word, add to return list
            word_indexes.append(self.word_index[word])
        if len(word_indexes)>self.max_len:
            self.max_len = len(word_indexes)
        return word_indexes
    
    def get_word_index(self, word):
        if word in self.word_index:
            return self.word_index[word]
        else:
            return 0
    
    
    # Different than P2, here we 
    def read_corpus(self, is_training):
        if is_training is True:
            target = train_list
        else:
            target = test_list
        print("reading corpus ...")
        for rate, text in tqdm(target):
            input = text.split(" ")
            tuple = (self.insert_fields(input), rate)
            if is_training: 
                self.training_reviews.append(tuple)
            else:
                self.test_reviews.append(tuple)
                    
                
# Inherient Dataset, convert list and int to tensors, load to GPU.
class ReviewRateDataset(Dataset):
   
    def __init__(self, review_rate_pairs): # NB: sequence_pairs is corpora.training_reviews, 
        # list of (sentence_list, rate)
        self.review_rate_pairs = review_rate_pairs

    def __len__(self):
        return len(self.review_rate_pairs)

    def __getitem__(self, idx):
        sentence_list, rate = self.review_rate_pairs[idx] 
        return torch.tensor(sentence_list), torch.tensor(int(rate))

# NB! This class will be in DataLoader function as a parameter for batch_sampler
class SortedBatchSampler(Sampler):
    """
        Each sequence in a mini-batch must of the same lengths, while our sentences
        are of various lengths.
        We can pad the sentences to the same lengths in each mini-batch.
        But if a short and long sentences are in the same mini-batch, more paddings
        are needed.
        We sort the sentences based on their lengths (in descending order)
            and then put sentences with similar lengths in a batch to reduce the paddings.
    """
    def __init__(self, dataset, batch_size):
        """
            dataset: an torch.utils.data.DataSet object containing all training sequences
            batch_size: the number of sequences to put in a mini-batch
        """

        # The sorting and batching go within this function.      
        self.batch_size = batch_size 
        # Sort the dataset (Based on the length of sentence.)
        dataset.review_rate_pairs  = sorted(dataset.review_rate_pairs,key=lambda x:len(x[0]), reverse=True)
        self.sorted_lengths = len(dataset)
        # Batching: Split the dataset into a list of datasets
        self.index_batches = []  
        # -- NB: Collate function does not work, so I pad it directly.
        for i in range(self.__len__()):
            self.index_batches.append(padding_collate_func(ReviewRateDataset(dataset.review_rate_pairs[i*batch_size:i*batch_size+batch_size])))
        # Now, each mini-batches is a ReviewRateDataset object
        # If else format is needed, may change it latter.
        
    def __iter__(self):
        """
            return a Python iterator object that iterates the mini-batchs of
                training data indices (not individual indices)
        """
        return iter(self.index_batches)

    def __len__(self):
        return self.sorted_lengths // self.batch_size

# NB! This function will be in DataLoader function as a parameter for collate_fn
def padding_collate_func(batch):
    """
        Transform pairs of input-output sequences in the batch to be of the same length using the function
            torch.nn.utils.rnn.pad_sequence.
        batch: An iterator and each element is a pair of (input_sequence, output_sequence).
        For POS tagging, len(input_sequence) = len(output_sequence). But for different
        pairs in batch, their lengths can differ.

        Example: a batch of 3 pairs of input/output sequences
                [([1,2,3],[1,1,1]), ([1,2,3,4],[2,2,2,2]), ([1,2,3,4,5],[3,3,3,3,3])]
                Note: [] encloses tensors (not numpy arra ys)
                
                
                !!!!!NB QUESTION:  it is the inner [] that encloses tensors, right?
                Comment: Batch is an element of a Sampler (see test_p1.py, a l[0] is a batch)
                
                
        return: two tensors (one for input sequence batch and another for output sequence batch).
                These tensors are padded with zeros so that all sequences in the same batch
                are of the same length.
        Example: input_sequence_batch = [[1,2,3,0,0], [1,2,3,4,0], [1,2,3,4,5]],
                 output_sequence_batch = [[1,1,1,0,0], [2,2,2,2,0], [3,3,3,3,3]]

    """
    ### Your codes go here (5 points) ###
    # Hint: read the article linked at the top of this cell.
    
    # NOTe
    # len(batch[0][0]) == len(batch.sequence_pairs[0]) == the target value (the maximum length for each batch)
    # I fill it might be easier to pad sequence_pairs with tuple (0,0), but let's see.
    
    # Doubles the memory takes by this batch, which is not good
    new_l = []
    rate = []
    for i,j in batch.review_rate_pairs:
        try:
            new_l.append( torch.tensor(i))
            rate.append( torch.tensor(int(j)))
        except:
            print(i, '\t', j)
    padded = pad_sequence(new_l, batch_first=True, padding_value=0)
    #print("len ", len(padded), len(rate)) #BUG: NEED ZIP HERE
    obj = ReviewRateDataset(list(zip(padded,rate)))
    # NB: For now the outter [] is neither tensor nor list, it is an obj!
    # Change latter if necessary
    # Yes, let's change it to two tensors return
    ret1 = []
    ret2 = []
    for i in obj:
        ret1.append(i[0])
        ret2.append(i[1])
    ret1 = torch.stack(ret1)
    ret2 = torch.stack(ret2)
    return ret1, ret2

In [3]:

from torch import embedding, nn
# There is really nothing to be stored in this object.
# -- But wait, how about self.rnn and self.fc?
# -- NB: NOW, I assume that the nn keep weights from the inherentance,
# -- And these functions as LSTM and FC will use these weight correctly
class LSTMScoreAssigner(nn.Module):
    def __init__(self, input_dim, output_dim, emb_dim, hid_dim, n_layers, dropout, bidirectional):
        """
        :param input_dim: size of the vocabulary (number of unique tokens)
        :param output_dim: number of unique POS tags 
        :param emb_dim: embedding dimensionality of each token
        :param hid_dim: number of hidden neurons of a hidden state/cell
        :param n_layers: number of RNN layers (2 for faster training)
        :param dropout: dropout rate between 0 and 1at the embedding layer and rnn
        :param bidirectional: 1 if use bidirectional and 0 if don't
        """
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        vector_weights = [1.0/input_dim]*input_dim
        self.vector_weights = nn.Parameter(torch.tensor(vector_weights, requires_grad=True))

        # before output, there is a dropout (except the last layer)
        
        # -- Comment: this is a part of the analysis (last part of this hw)
        # -- It feels like I have no control on this bidirectional since it is part of nn library.
        if bidirectional == 0:
            self.rnn = nn.LSTM(input_size = emb_dim, hidden_size = hid_dim, num_layers = n_layers, dropout=dropout)
            self.fc = nn.Linear(hid_dim, output_dim)
            self.num_directions = 1
        elif bidirectional == 1:
            self.rnn = nn.LSTM(input_size = emb_dim, hidden_size = hid_dim, num_layers = n_layers, dropout=dropout, bidirectional=True)
            self.fc = nn.Linear(hid_dim * 2, output_dim)
            self.num_directions = 2

        self.dropout = nn.Dropout(dropout)

    # -- COMMENT
    # -- The src means sourse, which is a 2d array batch_size by sentence_len, it is a big 2d tensor
    # -- NBBBBB: How to turn the POSTaggedDataset into a big 2d tensor see test_p2 line 4-7
    def forward(self, src):
        """

        :param src: a [batch_size, sentence_len] array.
                     Each row is a sequence of word indices and each column represents a position in the sequence.
        :return: the predicted logits at each position. 
        """

        emb = self.embedding(src)
        output = self.rnn(emb)[0]
        logit = self.fc(output)
        logit = torch.swapaxes(logit, 1, 2)
        weight_vector = self.vector_weights[src]
        weight_vector = torch.softmax(weight_vector.unsqueeze(dim=2),dim=1)
        
        return torch.bmm(logit,weight_vector).squeeze(2)



In [4]:

from torch import optim
import time
import math

BATCH_SIZE = 32
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

corpora = Corpora()

corpora.read_corpus(True)
corpora.read_corpus(False)

print(f'Number of training sentences = {len(corpora.training_reviews)}')
print(f'Number of test sentences = {len(corpora.test_reviews)}')
print(f'Number of unique input tokens = {len(corpora.word_index)}')
print(f'Maximal sentence length = {corpora.max_len}')

print("\n\n Creating training Dataset, Sampler, and Iterators...")
training_dataset = ReviewRateDataset(corpora.training_reviews)
training_sampler = SortedBatchSampler(training_dataset, batch_size=BATCH_SIZE)
training_iterator = DataLoader(training_dataset,
                                  collate_fn = padding_collate_func,
                                  batch_sampler = training_sampler)
print("\n\n Creating test Dataset, Sampler, and Iterators")
test_dataset = ReviewRateDataset(corpora.test_reviews)
test_sampler = SortedBatchSampler(test_dataset, batch_size=BATCH_SIZE)
test_iterator = DataLoader(test_dataset,
                              collate_fn = padding_collate_func,
                              batch_sampler = test_sampler)

print(f'Training first batch max length = {len(list(training_sampler)[0][0][0])}')
print(f'Training second batch max length = {len(list(training_sampler)[1][0][0])}')
print(f'Training last batch max length = {len(list(training_sampler)[-1][0][0])}')
print(f'Training second last batch max length = {len(list(training_sampler)[-2][0][0])}')



INPUT_DIM = len(corpora.word_index)
OUTPUT_DIM = 5
EMB_DIM = 64
HID_DIM = 64
N_LAYERS = 1 # number of LSTM layers.
BIDIRECT = 1 # 0: single direction (the default setting); 1: bidirectional
DROPOUT = 0.5
# initialize the model
ScoreAssigner = LSTMScoreAssigner(INPUT_DIM, OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DROPOUT, BIDIRECT).to(device)



# Glove Embedding here?
def init_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

ScoreAssigner.apply(init_weights)

optimizer = optim.Adam(ScoreAssigner.parameters())

# we use 0 to represent padded POS tags and the loss function should ignore that.
# we calculate the sum of losses of pairs in each batch
PAD_INDEX = 0


# input: vector of [length, output_dim], integer (score)
# criterion = nn.CrossEntropyLoss(reduction = 'sum', ignore_index = PAD_INDEX)

criterion = nn.CrossEntropyLoss(reduction = 'sum')

N_EPOCHS = 1
CLIP = 1

best_test_loss = float('inf')

training_losses = []
test_losses = []


reading corpus ...


100%|██████████| 4000/4000 [00:00<00:00, 48154.90it/s]


reading corpus ...


100%|██████████| 1000/1000 [00:00<00:00, 31238.76it/s]

Number of training sentences = 4000
Number of test sentences = 1000
Number of unique input tokens = 12426
Maximal sentence length = 875


 Creating training Dataset, Sampler, and Iterators...



C:\Users\klaud\AppData\Local\Temp/ipykernel_13476/115847253.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(sentence_list), torch.tensor(int(rate))




 Creating test Dataset, Sampler, and Iterators
Training first batch max length = 875
Training second batch max length = 228
Training last batch max length = 9
Training second last batch max length = 10


C:\Users\klaud\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -- The Iterator is a Dataloader object. 
# -- Use for loop in iterator.batch_sampler to access each batches
# -- In this case, each batches is having length 128

# -- Need to Figure out: The way to compute loss for RNN
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0
    num_batchs = 0
    total = 0

    # batch[0]: the word batch
    # batch[1]: the tag batch (target)
    print("training ...")
    for i, batch in tqdm(enumerate(iterator.batch_sampler)):
#         #skip first batch
#         if i == 0:
#             continue
        num_batchs += 1
        z = ScoreAssigner.forward(batch[0].to(device))
        #a = torch.softmax(z,dim=-1)
        loss = 0
        # softmax of logit
        d = torch.softmax(z,dim=-1)
        # cross entropy loss of softmax and score
        loss=criterion(d,(batch[1]-1).to(device))/BATCH_SIZE
        loss.backward()
        # Clips gradient norm of an iterable of parameters.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        total += 1

    return epoch_loss /total

confusion_matrix = torch.zeros(5,5)

def evaluate(model, iterator, criterion):

    model.eval()
    epoch_loss = 0
    num_epochs = 0
    total = 0

    for i, batch in tqdm(enumerate(iterator.batch_sampler)):
        num_epochs += 1
        z = ScoreAssigner.forward(batch[0].to(device))
        loss = 0
        # softmax of logit
        d = torch.softmax(z,dim=-1)
        # cross entropy loss of softmax and score
        loss=criterion(d,(batch[1]-1).to(device))/BATCH_SIZE
        epoch_loss += loss.item()
        total += 1
        
        # Load in confusion_matrix

        for i in range(len(d)):
            row = batch[1][i]-1
            col = torch.argmax(d[i])
            confusion_matrix[row][col] += 1
        
    return epoch_loss/total

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs




In [6]:
for epoch in range(N_EPOCHS):  
    print("epoch start: ", epoch)  
    start_time = time.time()
    training_loss = train(ScoreAssigner, training_iterator, optimizer, criterion, CLIP)
    training_losses.append(training_loss)
    test_loss = evaluate(ScoreAssigner, test_iterator, criterion)
    test_losses.append(test_loss)  
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if test_loss < best_test_loss:
        best_test_loss = test_loss 
        torch.save(ScoreAssigner.state_dict(), 'best_model.pt')
        

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s', end='')
    print(f'\tTrain Loss: {training_loss:.3f} | Test Loss: {test_loss:.3f}')
'''
import pickle
with open(f'results/losses_L{N_LAYERS}_D{DROPOUT}_B{BIDIRECT}.pkl', 'wb') as f:
    pickle.dump({'training_losses': training_losses,
                'test_losses': test_losses}, f)
'''

epoch start:  0
training ...


125it [00:07, 16.71it/s]
31it [00:00, 35.72it/s]

Epoch: 01 | Time: 0m 8s	Train Loss: 1.602 | Test Loss: 1.574


"\nimport pickle\nwith open(f'results/losses_L{N_LAYERS}_D{DROPOUT}_B{BIDIRECT}.pkl', 'wb') as f:\n    pickle.dump({'training_losses': training_losses,\n                'test_losses': test_losses}, f)\n"

In [7]:

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


sns.set()
x = np.arange(len(training_losses))
plt.plot(x, training_losses, label = 'training loss')
plt.plot(x, test_losses, label = 'test loss')
plt.legend()

ModuleNotFoundError: No module named 'seaborn'

In [72]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Get the set of English stopwords to remove from reviews
stopset = set(stopwords.words('english'))
# Do not include 'not' as a stop word because it can have a strong impact on the meaning of a sentence
stopset.remove('not')
tokenizer = RegexpTokenizer("[a-zA-Z]+")

def clean_text(review):
    review = review.lower()
    # Split line into tokens
    review = tokenizer.tokenize(review)
    # Remove stop words
    review = [i for i in review if i not in stopset]
    # Combine lines back into new string
    return review

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\klaud\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
examples = [
    "amazing",
    "good",
    "okay",
    "terrible",
    "i loved the lotion! it is great quality and makes my skin feel amazing!",
    "i hate it. it is cheap and broke as soon as i used it. i will never buy anything from this company again!"
]
for e in examples:
    tensor = clean_text(e)
    tensor = [corpora.get_word_index(word) for word in tensor]
    tensor = torch.tensor([tensor]).to(device)
    z = ScoreAssigner.forward(tensor)
    d = torch.softmax(z,dim=-1)
    pred = torch.sum(d / torch.sum(d) * torch.tensor([1, 2, 3, 4, 5]))
    print("Example: ", e, "\nPredicted Score: ", pred.item(), "\nRNN output: ", d)
    print()


Example:  amazing 
Predicted Score:  4.555298805236816 
RNN output:  tensor([[0.0141, 0.0221, 0.0149, 0.2921, 0.6568]], grad_fn=<SoftmaxBackward0>)

Example:  good 
Predicted Score:  4.201083183288574 
RNN output:  tensor([[0.0191, 0.0851, 0.0343, 0.3987, 0.4628]], grad_fn=<SoftmaxBackward0>)

Example:  okay 
Predicted Score:  2.2445626258850098 
RNN output:  tensor([[0.4372, 0.1223, 0.2760, 0.0876, 0.0769]], grad_fn=<SoftmaxBackward0>)

Example:  terrible 
Predicted Score:  1.2125855684280396 
RNN output:  tensor([[0.8958, 0.0044, 0.0944, 0.0021, 0.0032]], grad_fn=<SoftmaxBackward0>)

Example:  i loved the lotion! it is great quality and makes my skin feel amazing! 
Predicted Score:  4.393971920013428 
RNN output:  tensor([[0.0202, 0.0438, 0.0271, 0.3396, 0.5693]], grad_fn=<SoftmaxBackward0>)

Example:  i hate it. it is cheap and broke as soon as i used it. i will never buy anything from this company again! 
Predicted Score:  1.4325823783874512 
RNN output:  tensor([[0.7982, 0.0166, 0

In [65]:
print(confusion_matrix)

tensor([[147.,   0.,   0.,  39.,   2.],
        [145.,   0.,   0.,  56.,   5.],
        [122.,   0.,   0.,  67.,   6.],
        [ 76.,   0.,   0., 116.,   9.],
        [ 37.,   0.,   0., 137.,  28.]])


In [11]:
top50 = torch.argsort(ScoreAssigner.vector_weights)[0:50]
top50_words = []
for i in top50:
    for j in corpora.word_index.keys():
        if corpora.word_index[j] == i:
            top50_words.append(j)
            continue
            
top50_words

['spend',
 'health',
 'thing',
 'looks',
 'another',
 'sign',
 'trouble',
 'machines',
 'regimen',
 'eyebrows',
 'problem',
 'measure',
 'fire',
 'hammer',
 'intended',
 'keep',
 'currently',
 'walmart',
 'excited',
 'affect',
 'scrape',
 'pool',
 'smears',
 'fine',
 'stream',
 'olay',
 'internal',
 'lacks',
 'necessary',
 'numerous',
 'truly',
 'recomend',
 'yes',
 'entire',
 'tacky',
 'less',
 'dual',
 'shocked',
 'mainly',
 'turning',
 'whereas',
 'drinks',
 'sounded',
 'severity',
 'appropriate',
 'pedicure',
 'kill',
 'tightening',
 'regular',
 'breath']

In [12]:
distance = 0
for i in range(5):
    for j in range(5):
        distance+= confusion_matrix[i][j]*abs(i-j)
print(distance)

tensor(1242.)
